## Portfolio Exercise Starbucks
<br>

<img src="https://opj.ca/wp-content/uploads/2018/02/New-Starbucks-Logo-1200x969.jpg" width="200" height="200">
<br>
<br>
 
#### Background Information

The dataset you will be provided in this portfolio exercise was originally used as a take-home assignment provided by Starbucks for their candidates.   The data for this exercise consists of about 120,000 data points split in 2:1 ratio among training and test files. Each data point includes one column indicating whether or not an individual was sent a promotion for a specific product, and one column indicating whether or not that individual eventually purchased that product. Each individual also had seven additional features associated with them.

#### Optimization Strategy

Your task is to use the training data to understand what patterns in V1-V7 to indicate that a promotion should be provided to a user.  Specifically, your goal is to maximize the following metrics:

* **Incremental Response Rate (IRR)** 

Ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group.

* **Net Incremental Revenue (NIR)**

The total number of purchasers that received the promotion times 10 minus the number of promotions given times 0.15 minus the number of purchasers who were not given the promotion times 10.

For a full description of what starbucks provides to candidates see the [instructions available here](https://drive.google.com/open?id=18klca9Sef1Rs6q8DW4l7o349r8B70qXM).

Below you can find the training data provided.  Explore the data and different optimization strategies.

#### How To Test Your Strategy?

When you feel like you have an optimization strategy, complete the **promotion_strategy** function to pass to the **test_results** function.  
From past data, we know there are four possible outomes:

Table of actual promotion vs. predicted promotion customers.  

|   | Actual      |    |    | 
|---|-------------|----|----|
| **Predicted**   | Yes| No |  
| Yes             | **I**   | **II**|  
| No              | **III** | **IV**|  

The metrics are only being compared for the individual's we predict should obtain the promotion - that is quadrants I and II here.  Since the first set of individuals that receive the promotion (in the training set) receive it randomly, we can expect that quadrants I and II will have approximately equal participants.  

Comparing quadrant I to II then gives an idea of how well your promotion strategy will work in the future. 


Get started by reading in the data below.  See how each variable or combination of variables along with a promotion influences the chance of purchasing.  When you feel like you have a strategy for who should receive a promotion, test your strategy against the test dataset used in the final test_results function.

In [24]:
# load in packages
from itertools import combinations

from test_results import test_results, score
import numpy as np
import pandas as pd
#import scipy as sp
import scipy.stats as stats
import sklearn as sk
from statsmodels.stats import proportion as proptests

import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

# load in the data
train_data = pd.read_csv('data/training.csv')
train_data.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2


In [5]:
train_data.shape

(84534, 10)

In [ ]:
# Cells for you to work and document as necessary - 
# definitely feel free to add more cells as you need

## Analyze the result of the experinment and identify effect of the treatment for key metrics 

- IRR
- NIR


In [8]:
# explorative analysis
train_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84534 entries, 0 to 84533
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         84534 non-null  int64  
 1   Promotion  84534 non-null  object 
 2   purchase   84534 non-null  int64  
 3   V1         84534 non-null  int64  
 4   V2         84534 non-null  float64
 5   V3         84534 non-null  float64
 6   V4         84534 non-null  int64  
 7   V5         84534 non-null  int64  
 8   V6         84534 non-null  int64  
 9   V7         84534 non-null  int64  
dtypes: float64(2), int64(7), object(1)
memory usage: 6.4+ MB


In [13]:
# treatment group and control group 
# first, make sure two groups are approximately the same 

train_t = train_data[train_data['Promotion'] == 'Yes']
train_c = train_data[train_data['Promotion'] == 'No']
print(train_t.shape[0],train_c.shape[0])

42364 42170


In [ ]:
# Visualize 



In [16]:
# test if proportion of two groups are approximately 0.5 vs 0.5
def proportion_test(count,nobs):
    p_val = proptests.binom_test(
        count+0.5,
        nobs,
        prop =0.5,
        alternative = 'two-sided'
    )
    
    if p_val*2 < 0.05:
        print(f'P-value: {round(p_val,3)} - Statistically significant; reject H0.')
    else:
        print(f'P-value: {round(p_val,3)} - Fail to reject H0; two groups have roughly same # of observations.')

proportion_test(train_t.shape[0], train_data.shape[0])

P-value: 0.507 - Fail to reject H0; two groups have roughly same # of observations.


In [25]:
# Identified promotion's effect on IRR 

# Ho: IRRs of treatment group and control group has no difference 

x_treat = train_t['purchase'].sum() 
x_cont = train_c['purchase'].sum() 
n_treat = train_t.shape[0]
n_cont = train_c.shape[0]
N = train_data.shape[0]

def prop_diff_test(x_cont, x_exp, n_cont, n_exp, N, alpha = 0.05, diff_null = 0):

    p_cont = x_cont / n_cont
    p_exp = x_exp / n_exp
    p_pool = (x_cont+x_exp)/N 
    
    se_pool = np.sqrt(
        (p_pool*(1-p_pool))*((1/n_cont)+(1/n_exp))
    )

    z_test = ((p_exp - p_cont) - diff_null)/se_pool
    #p_val = stats.norm.sf(abs(z_test))
    p_val = 1-stats.norm.cdf(z_test)
    
    if p_val > alpha:
        print(f'P-value: {round(p_val,5)}\nZ-score: {round(z_test,3)}\nFail to reject H0; There is no significant difference.')
    else:
        print(f'P-value: {round(p_val,5)}\nZ-score: {round(z_test,3)}\nReject H0; There is a significant difference.')

prop_diff_test(x_cont, x_treat, n_cont, n_treat, N)

P-value: 0.0
Z-score: 12.468
Reject H0; There is a significant difference.


In [ ]:
# Identified promotion's effect on IRR 

# Ho: IRRs of treatment group and control group has no difference 

In [ ]:
def promotion_strategy(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''
    
    
    
    
    return promotion

In [ ]:
# This will test your results, and provide you back some information 
# on your how well your promotion_strategy will work in practice

test_results(promotion_strategy)